In [109]:
import sklearn

In [3]:
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [110]:
from imblearn.combine import SMOTETomek

In [111]:
#Required lib

import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc , accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [112]:
import warnings
warnings.filterwarnings("ignore")

In [113]:
df = pd.read_csv('copper_final.csv')

In [114]:
df

,quantity_tons,customer,country,status,item_type,application,thickness,width,product_ref,selling_price,item_date,delivery_date_pred,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year
0,3.991779,30156308,28.0,1,5.0,2.302585,0.693147,7.313220,1670798778,6.749931,2021-04-01,2021-07-01,1,4,2021,1,7,2021
1,1.259203,30209509,30.0,2,5.0,3.713572,-0.967584,7.025538,611993,5.975038,2021-04-01,2021-07-01,1,4,2021,1,7,2021
2,4.235147,30341428,38.0,1,3.0,2.302585,-0.510826,7.150701,1668701376,7.217443,2021-04-01,2021-07-01,1,4,2021,1,7,2021
3,2.216566,30209509,30.0,2,5.0,3.713572,-0.967584,7.025538,611993,5.975038,2021-04-01,2021-07-01,1,4,2021,1,7,2021
4,3.314642,30165529,78.0,1,5.0,2.302585,-0.287682,7.130899,164141591,7.001246,2021-04-01,2021-07-01,1,4,2021,1,7,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181633,4.629691,30200854,25.0,1,5.0,3.713572,-0.040822,7.106606,164141591,6.381816,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181634,5.337954,30200854,25.0,1,5.0,3.713572,-0.051293,7.313220,164141591,6.378426,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181635,1.443523,30200854,25.0,1,5.0,3.713572,-0.342490,7.130899,164141591,6.428105,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181636,-0.323075,30200854,25.0,1,5.0,3.713572,-0.162519,7.130899,164141591,6.398595,2020-07-02,2020-08-01,2,7,2020,1,8,2020


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181638 entries, 0 to 181637
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   quantity_tons        181638 non-null  float64
 1   customer             181638 non-null  int64  
 2   country              181638 non-null  float64
 3   status               181638 non-null  int64  
 4   item_type            181638 non-null  float64
 5   application          181638 non-null  float64
 6   thickness            181638 non-null  float64
 7   width                181638 non-null  float64
 8   product_ref          181638 non-null  int64  
 9   selling_price        181638 non-null  float64
 10  item_date            181638 non-null  object 
 11  delivery_date_pred   181638 non-null  object 
 12  item_date_day        181638 non-null  int64  
 13  item_date_month      181638 non-null  int64  
 14  item_date_year       181638 non-null  int64  
 15  delivery_date_day

In [116]:
df.columns

Index(['quantity_tons', 'customer', 'country', 'status', 'item_type',
       'application', 'thickness', 'width', 'product_ref', 'selling_price',
       'item_date', 'delivery_date_pred', 'item_date_day', 'item_date_month',
       'item_date_year', 'delivery_date_day', 'delivery_date_month',
       'delivery_date_year'],
      dtype='object')

In [117]:
df['status'].value_counts()

status
1    116008
0     34433
5     19568
7      4276
3      4168
2      3121
8        53
4        10
6         1
Name: count, dtype: int64

In [118]:
df = df[['quantity_tons', 'customer', 'country',  'item_type',
       'application', 'thickness', 'width', 'product_ref',
       'item_date_day', 'item_date_month',
       'item_date_year', 'delivery_date_day', 'delivery_date_month',
       'delivery_date_year','selling_price','status']]

In [119]:
df = df.query('status==1 or status ==0')
df

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price,status
0,3.991779,30156308,28.0,5.0,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931,1
2,4.235147,30341428,38.0,3.0,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443,1
4,3.314642,30165529,78.0,5.0,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246,1
5,3.473063,30202362,27.0,3.0,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384,1
7,3.035295,30211222,30.0,5.0,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181633,4.629691,30200854,25.0,5.0,3.713572,-0.040822,7.106606,164141591,2,7,2020,1,8,2020,6.381816,1
181634,5.337954,30200854,25.0,5.0,3.713572,-0.051293,7.313220,164141591,2,7,2020,1,8,2020,6.378426,1
181635,1.443523,30200854,25.0,5.0,3.713572,-0.342490,7.130899,164141591,2,7,2020,1,8,2020,6.428105,1
181636,-0.323075,30200854,25.0,5.0,3.713572,-0.162519,7.130899,164141591,2,7,2020,1,8,2020,6.398595,1


In [120]:
x = df.drop('status',axis=1)
y = df['status']

In [121]:
x

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price
0,3.991779,30156308,28.0,5.0,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931
2,4.235147,30341428,38.0,3.0,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443
4,3.314642,30165529,78.0,5.0,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246
5,3.473063,30202362,27.0,3.0,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384
7,3.035295,30211222,30.0,5.0,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181633,4.629691,30200854,25.0,5.0,3.713572,-0.040822,7.106606,164141591,2,7,2020,1,8,2020,6.381816
181634,5.337954,30200854,25.0,5.0,3.713572,-0.051293,7.313220,164141591,2,7,2020,1,8,2020,6.378426
181635,1.443523,30200854,25.0,5.0,3.713572,-0.342490,7.130899,164141591,2,7,2020,1,8,2020,6.428105
181636,-0.323075,30200854,25.0,5.0,3.713572,-0.162519,7.130899,164141591,2,7,2020,1,8,2020,6.398595


In [122]:
y

0         1
2         1
4         1
5         1
7         1
         ..
181633    1
181634    1
181635    1
181636    1
181637    1
Name: status, Length: 150441, dtype: int64

In [123]:
model = SMOTETomek()
x_new , y_new = model.fit_resample(x,y)
x_new.shape,y_new.shape

((231076, 15), (231076,))

In [124]:
y_new.value_counts(normalize=True)

status
1    0.5
0    0.5
Name: proportion, dtype: float64

In [125]:
x_new

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price
0,3.991779,30156308,28.0,5.000000,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931
1,4.235147,30341428,38.0,3.000000,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443
2,3.314642,30165529,78.0,5.000000,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246
3,3.473063,30202362,27.0,3.000000,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384
4,3.035295,30211222,30.0,5.000000,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231071,3.324186,30165529,78.0,5.000000,2.302585,2.185571,6.948113,640665,2,12,2020,1,1,2021,6.437630
231072,2.261184,30198074,26.0,3.000000,2.708050,1.791759,7.520313,640405,11,1,2021,1,6,2021,6.752847
231073,2.525402,30197559,25.0,5.000000,2.708050,1.766401,7.313220,640405,3,12,2020,1,4,2021,6.566345
231074,5.419820,30198507,26.0,4.339598,2.302585,0.972984,7.192189,1668701718,11,2,2021,1,7,2021,6.902705


In [126]:
y_new

0         1
1         1
2         1
3         1
4         1
         ..
231071    0
231072    0
231073    0
231074    0
231075    0
Name: status, Length: 231076, dtype: int64

In [127]:
x_new['status'] = y_new
x_new.to_csv('classification_dataset.csv',index = False)

In [128]:
x_new

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price,status
0,3.991779,30156308,28.0,5.000000,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931,1
1,4.235147,30341428,38.0,3.000000,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443,1
2,3.314642,30165529,78.0,5.000000,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246,1
3,3.473063,30202362,27.0,3.000000,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384,1
4,3.035295,30211222,30.0,5.000000,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231071,3.324186,30165529,78.0,5.000000,2.302585,2.185571,6.948113,640665,2,12,2020,1,1,2021,6.437630,0
231072,2.261184,30198074,26.0,3.000000,2.708050,1.791759,7.520313,640405,11,1,2021,1,6,2021,6.752847,0
231073,2.525402,30197559,25.0,5.000000,2.708050,1.766401,7.313220,640405,3,12,2020,1,4,2021,6.566345,0
231074,5.419820,30198507,26.0,4.339598,2.302585,0.972984,7.192189,1668701718,11,2,2021,1,7,2021,6.902705,0


In [129]:
df = pd.read_csv('classification_dataset.csv')

In [130]:
df

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price,status
0,3.991779,30156308,28.0,5.000000,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931,1
1,4.235147,30341428,38.0,3.000000,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443,1
2,3.314642,30165529,78.0,5.000000,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246,1
3,3.473063,30202362,27.0,3.000000,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384,1
4,3.035295,30211222,30.0,5.000000,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231071,3.324186,30165529,78.0,5.000000,2.302585,2.185571,6.948113,640665,2,12,2020,1,1,2021,6.437630,0
231072,2.261184,30198074,26.0,3.000000,2.708050,1.791759,7.520313,640405,11,1,2021,1,6,2021,6.752847,0
231073,2.525402,30197559,25.0,5.000000,2.708050,1.766401,7.313220,640405,3,12,2020,1,4,2021,6.566345,0
231074,5.419820,30198507,26.0,4.339598,2.302585,0.972984,7.192189,1668701718,11,2,2021,1,7,2021,6.902705,0


In [131]:
x = df.drop('status',axis=True)
y = df['status']
x

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,selling_price
0,3.991779,30156308,28.0,5.000000,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,6.749931
1,4.235147,30341428,38.0,3.000000,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,7.217443
2,3.314642,30165529,78.0,5.000000,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,7.001246
3,3.473063,30202362,27.0,3.000000,2.302585,-0.510826,6.917706,628377,1,4,2021,1,7,2021,6.880384
4,3.035295,30211222,30.0,5.000000,2.302585,-0.798508,6.802395,611993,1,4,2021,1,7,2021,7.181736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231071,3.324186,30165529,78.0,5.000000,2.302585,2.185571,6.948113,640665,2,12,2020,1,1,2021,6.437630
231072,2.261184,30198074,26.0,3.000000,2.708050,1.791759,7.520313,640405,11,1,2021,1,6,2021,6.752847
231073,2.525402,30197559,25.0,5.000000,2.708050,1.766401,7.313220,640405,3,12,2020,1,4,2021,6.566345
231074,5.419820,30198507,26.0,4.339598,2.302585,0.972984,7.192189,1668701718,11,2,2021,1,7,2021,6.902705


In [132]:
y

0         1
1         1
2         1
3         1
4         1
         ..
231071    0
231072    0
231073    0
231074    0
231075    0
Name: status, Length: 231076, dtype: int64

In [133]:
def model_classification(x,y,algorithm):
    for i in algorithm:
        # Split data
        xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3)

        # Model
        model = i().fit(xtrain,ytrain)

        # Predict for train and test accuracy

        y_train_pred = model.predict(xtrain)
        y_test_pred  = model.predict(xtest)

        # Accuracy score

        training = accuracy_score(ytrain,y_train_pred)
        testing = accuracy_score(ytest,y_test_pred)
        data = {'Algorithm':i.__name__, 'Training Acuuracy':training,'Testing Accuracy':testing}

        print(data)

In [134]:
model_classification(x,y,[DecisionTreeClassifier,ExtraTreesClassifier,RandomForestClassifier,XGBClassifier])

{'Algorithm': 'DecisionTreeClassifier', 'Training Acuuracy': 1.0, 'Testing Accuracy': 0.9633166481542922}
{'Algorithm': 'ExtraTreesClassifier', 'Training Acuuracy': 1.0, 'Testing Accuracy': 0.9848823622751468}
{'Algorithm': 'RandomForestClassifier', 'Training Acuuracy': 1.0, 'Testing Accuracy': 0.9810596771634233}
{'Algorithm': 'XGBClassifier', 'Training Acuuracy': 0.9537381068666424, 'Testing Accuracy': 0.9462516048064856}


In [135]:
param_grid = {
    'n_estimators': [50, 100, 150],           # Number of trees in the forest
    'criterion': ['gini', 'entropy'],       # Split criterion
    'max_depth': [None, 10, 20, 30],        # Maximum depth of trees
    'min_samples_split': [2, 5, 10],       # Min samples to split an internal node
    'min_samples_leaf': [1, 2, 4],         # Min samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider
    'class_weight': [None, 'balanced'],    # Class weight
    'random_state': [42],                  # Random seed for reproducibility
}

In [136]:
#extra_tree = ExtraTreesClassifier()
#grid_search = GridSearchCV(extra_tree, param_grid, cv=5, n_jobs=-1)
#grid_search.fit(x_new, y_new)
# print("Best Hyperparameters:", grid_search.best_params_)
     

In [137]:
# grid_search.best_score_

In [138]:
# Model building with Best Parameters

xtrain,xtest,ytrain,ytest = train_test_split(x.values,y,test_size=0.2)

# Model
model = ExtraTreesClassifier().fit(xtrain, ytrain)

# predicition
y_pred_train = model.predict(xtrain)
y_pred_test = model.predict(xtest)
accuracy_train = metrics.accuracy_score(ytrain, y_pred_train)
accuracy_test = metrics.accuracy_score(ytest, y_pred_test)
('Training accuracy :',accuracy_train),( 'Testing Accuracy :', accuracy_test)

(('Training accuracy :', 1.0), ('Testing Accuracy :', 0.9855677687380994))

In [139]:
x.values

array([[3.99177900e+00, 3.01563080e+07, 2.80000000e+01, ...,
        7.00000000e+00, 2.02100000e+03, 6.74993119e+00],
       [4.23514731e+00, 3.03414280e+07, 3.80000000e+01, ...,
        7.00000000e+00, 2.02100000e+03, 7.21744343e+00],
       [3.31464207e+00, 3.01655290e+07, 7.80000000e+01, ...,
        7.00000000e+00, 2.02100000e+03, 7.00124562e+00],
       ...,
       [2.52540224e+00, 3.01975590e+07, 2.50000000e+01, ...,
        4.00000000e+00, 2.02100000e+03, 6.56634491e+00],
       [5.41982009e+00, 3.01985070e+07, 2.60000000e+01, ...,
        7.00000000e+00, 2.02100000e+03, 6.90270537e+00],
       [2.20237834e+00, 3.03976320e+07, 7.80000000e+01, ...,
        9.00000000e+00, 2.02000000e+03, 6.41865702e+00]])

In [140]:
model

ExtraTreesClassifier()

In [141]:
df.columns

Index(['quantity_tons', 'customer', 'country', 'item_type', 'application',
       'thickness', 'width', 'product_ref', 'item_date_day', 'item_date_month',
       'item_date_year', 'delivery_date_day', 'delivery_date_month',
       'delivery_date_year', 'selling_price', 'status'],
      dtype='object')

In [40]:
quantity_tons = float(input('Enter the Quantity Ton value :'))

customer = float(input('Enter the customer code :'))

country = float(input('Enter the country code :'))

item_type = float(input('Enter the item type value :'))

application = float(input('Enter the application value :'))

thickness = float(input('Enter the thickness value :'))

width = float(input('Enter the width value :'))

product_ref = float(input('Enter the product ref value :'))

item_date_day = float(input('Enter the item day :'))

item_date_month = float(input('Enter the item month:'))

item_date_year = float(input('Enter the item year :'))

delivery_date_day = float(input('Enter the delivery day :'))

delivery_date_month = float(input('Enter the delivery month :'))

delivery_date_year = float(input('Enter the delivery year :'))

selling_price = float(input('Enter the selling price :'))

Enter the Quantity Ton value :5.2336561533
Enter the customer code :286585
Enter the country code :20
Enter the item type value :4
Enter the application value :2.369955858
Enter the thickness value :1.026562965
Enter the width value :6.56296258626
Enter the product ref value :16549586
Enter the item day :25
Enter the item month:4
Enter the item year :2021
Enter the delivery day :1
Enter the delivery month :10
Enter the delivery year :2021
Enter the selling price :6.24564625632


In [142]:
x = df.loc[5000]

x.values.tolist()

[3.800216003086149,
 30205825.0,
 25.0,
 5.0,
 2.302585092994046,
 -0.3566749439387324,
 6.907755278982137,
 1332077137.0,
 24.0,
 3.0,
 2021.0,
 1.0,
 8.0,
 2021.0,
 6.966024187106113,
 1.0]

In [143]:
x

quantity_tons          3.800216e+00
customer               3.020582e+07
country                2.500000e+01
item_type              5.000000e+00
application            2.302585e+00
thickness             -3.566749e-01
width                  6.907755e+00
product_ref            1.332077e+09
item_date_day          2.400000e+01
item_date_month        3.000000e+00
item_date_year         2.021000e+03
delivery_date_day      1.000000e+00
delivery_date_month    8.000000e+00
delivery_date_year     2.021000e+03
selling_price          6.966024e+00
status                 1.000000e+00
Name: 5000, dtype: float64

In [161]:
# Provided data for prediction

# Lost

# p = [5.0550630558573335,27,3,2.302585092994046,1.0986122886681098,7.313220387090301,1670798778.0,28.0,3,2021,1,7,2021,6.891625897052253,
#  ]

# WON

p = [quantity_tons, customer ,country, item_type, application,
       thickness, width, product_ref, item_date_day, item_date_month,
       item_date_year, delivery_date_day, delivery_date_month,
       delivery_date_year,
       selling_price]  # 0

# prediction

In [145]:
#model predection
x = model.predict([[3.824247867319961,
 30205825.0,
 25.0,
 5.0,
 2.302585092994046,
 0.6931471805599453,
 7.098375638590786,
 1332077137.0,
 24.0,
 3.0,
 2021.0,
 1.0,
 8.0,
 2021.0,
 6.907755278982137]])

if x[0]==1:
    print()
    print('WON')
    print()
else :
    print()
    print('LOST')
    print()


WON



# Download the Pickle Model:

In [146]:
with open('classification_dataset.pkl','wb') as f:
      pickle.dump(model,f)

In [147]:
with open('classification_dataset.pkl', 'rb') as f:
    model = pickle.load(f)

In [148]:
model

ExtraTreesClassifier()

In [160]:
x = model.predict([p])

if x[0]==1:
    print()
    print('WON')
    print()
else :
    print()
    print('LOST')
    print()

# Regression model

In [150]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pickle

In [151]:
import pandas as pd

df = pd.read_csv('copper_final.csv')
df

,quantity_tons,customer,country,status,item_type,application,thickness,width,product_ref,selling_price,item_date,delivery_date_pred,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year
0,3.991779,30156308,28.0,1,5.0,2.302585,0.693147,7.313220,1670798778,6.749931,2021-04-01,2021-07-01,1,4,2021,1,7,2021
1,1.259203,30209509,30.0,2,5.0,3.713572,-0.967584,7.025538,611993,5.975038,2021-04-01,2021-07-01,1,4,2021,1,7,2021
2,4.235147,30341428,38.0,1,3.0,2.302585,-0.510826,7.150701,1668701376,7.217443,2021-04-01,2021-07-01,1,4,2021,1,7,2021
3,2.216566,30209509,30.0,2,5.0,3.713572,-0.967584,7.025538,611993,5.975038,2021-04-01,2021-07-01,1,4,2021,1,7,2021
4,3.314642,30165529,78.0,1,5.0,2.302585,-0.287682,7.130899,164141591,7.001246,2021-04-01,2021-07-01,1,4,2021,1,7,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181633,4.629691,30200854,25.0,1,5.0,3.713572,-0.040822,7.106606,164141591,6.381816,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181634,5.337954,30200854,25.0,1,5.0,3.713572,-0.051293,7.313220,164141591,6.378426,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181635,1.443523,30200854,25.0,1,5.0,3.713572,-0.342490,7.130899,164141591,6.428105,2020-07-02,2020-08-01,2,7,2020,1,8,2020
181636,-0.323075,30200854,25.0,1,5.0,3.713572,-0.162519,7.130899,164141591,6.398595,2020-07-02,2020-08-01,2,7,2020,1,8,2020


In [152]:
df.columns

Index(['quantity_tons', 'customer', 'country', 'status', 'item_type',
       'application', 'thickness', 'width', 'product_ref', 'selling_price',
       'item_date', 'delivery_date_pred', 'item_date_day', 'item_date_month',
       'item_date_year', 'delivery_date_day', 'delivery_date_month',
       'delivery_date_year'],
      dtype='object')

In [153]:
df = df[['quantity_tons', 'customer', 'country', 'item_type',
       'application', 'thickness', 'width', 'product_ref',
        'item_date_day', 'item_date_month',
       'item_date_year', 'delivery_date_day', 'delivery_date_month',
       'delivery_date_year', 'status','selling_price']]
     

In [154]:
x = df.drop('selling_price',axis=1)
y = df['selling_price']
x

,quantity_tons,customer,country,item_type,application,thickness,width,product_ref,item_date_day,item_date_month,item_date_year,delivery_date_day,delivery_date_month,delivery_date_year,status
0,3.991779,30156308,28.0,5.0,2.302585,0.693147,7.313220,1670798778,1,4,2021,1,7,2021,1
1,1.259203,30209509,30.0,5.0,3.713572,-0.967584,7.025538,611993,1,4,2021,1,7,2021,2
2,4.235147,30341428,38.0,3.0,2.302585,-0.510826,7.150701,1668701376,1,4,2021,1,7,2021,1
3,2.216566,30209509,30.0,5.0,3.713572,-0.967584,7.025538,611993,1,4,2021,1,7,2021,2
4,3.314642,30165529,78.0,5.0,2.302585,-0.287682,7.130899,164141591,1,4,2021,1,7,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181633,4.629691,30200854,25.0,5.0,3.713572,-0.040822,7.106606,164141591,2,7,2020,1,8,2020,1
181634,5.337954,30200854,25.0,5.0,3.713572,-0.051293,7.313220,164141591,2,7,2020,1,8,2020,1
181635,1.443523,30200854,25.0,5.0,3.713572,-0.342490,7.130899,164141591,2,7,2020,1,8,2020,1
181636,-0.323075,30200854,25.0,5.0,3.713572,-0.162519,7.130899,164141591,2,7,2020,1,8,2020,1


In [155]:
y

0         6.749931
1         5.975038
2         7.217443
3         5.975038
4         7.001246
            ...   
181633    6.381816
181634    6.378426
181635    6.428105
181636    6.398595
181637    6.408529
Name: selling_price, Length: 181638, dtype: float64

In [156]:
def model_regression(x,y,algorithm):
    for i in algorithm:
        xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)
        model = i().fit(xtrain,ytrain)
        # predict for train and test accuracy
        y_train_pred = model.predict(xtrain)
        y_test_pred  = model.predict(xtest)

       # Accuracy score
        training = r2_score(ytrain,y_train_pred)
        testing = r2_score(ytest,y_test_pred)
        data = {'Algorithm':i.__name__, 'Training R2 Score':training,'Testing R2 Score':testing}
        print(data)

In [157]:
model_regression(x,y,[DecisionTreeRegressor,ExtraTreesRegressor,RandomForestRegressor,XGBRegressor])

{'Algorithm': 'DecisionTreeRegressor', 'Training R2 Score': 0.9999198376368532, 'Testing R2 Score': 0.922921981148992}
{'Algorithm': 'ExtraTreesRegressor', 'Training R2 Score': 0.9999129151775097, 'Testing R2 Score': 0.9536609714916936}
{'Algorithm': 'RandomForestRegressor', 'Training R2 Score': 0.993965138133206, 'Testing R2 Score': 0.9571631429053296}
{'Algorithm': 'XGBRegressor', 'Training R2 Score': 0.9506547239318058, 'Testing R2 Score': 0.945248207988052}


# Hyper parameter Tuning Using grid-search cv

In [159]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=42)
param_grid_r = {'max_depth'      : [2, 5, 10, 20],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf' : [1, 2, 4],
              'max_features'     : ['sqrt', 'log2', None]}

grid_search_r = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid_r, cv=5, n_jobs=-1)
grid_search_r.fit(xtrain, ytrain)

In [61]:
grid_search_r.best_params_,grid_search_r.best_score_

({'max_depth': 20,
  'max_features': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2},
 0.9527094116880852)

In [56]:
xtrain, xtest, ytrain, ytest = train_test_split(x.values,y,test_size=0.2)

model = ExtraTreesRegressor().fit(xtrain, ytrain)

y_pred_train = model.predict(xtrain)

y_pred_test = model.predict(xtest)

r2_train = r2_score(ytrain, y_pred_train)

r2_test = r2_score(ytest, y_pred_test)

r2_train, r2_test

# with normal df

(0.9999273605080216, 0.9564084406137079)

In [58]:
x.values

array([[ 3.99177900e+00,  3.01563080e+07,  2.80000000e+01, ...,
         7.00000000e+00,  2.02100000e+03,  1.00000000e+00],
       [ 1.25920293e+00,  3.02095090e+07,  3.00000000e+01, ...,
         7.00000000e+00,  2.02100000e+03,  2.00000000e+00],
       [ 4.23514731e+00,  3.03414280e+07,  3.80000000e+01, ...,
         7.00000000e+00,  2.02100000e+03,  1.00000000e+00],
       ...,
       [ 1.44352348e+00,  3.02008540e+07,  2.50000000e+01, ...,
         8.00000000e+00,  2.02000000e+03,  1.00000000e+00],
       [-3.23075153e-01,  3.02008540e+07,  2.50000000e+01, ...,
         8.00000000e+00,  2.02000000e+03,  1.00000000e+00],
       [ 6.00804271e+00,  3.02008540e+07,  2.50000000e+01, ...,
         8.00000000e+00,  2.02000000e+03,  1.00000000e+00]])

In [55]:
model

ExtraTreesRegressor()

In [63]:
quantity_tons = float(input('Enter the Quantity Ton value :'))

customer = float(input('enter the cusomer value'))

country = float(input('Enter the country code :'))

status = float(input('Enter the  status :'))

item_type = float(input('Enter the item type value :'))

application = float(input('Enter the application value :'))

thickness = float(input('Enter the thickness value :'))

width = float(input('Enter the width value :'))

product_ref = float(input('Enter the product ref value :'))

item_date_day = float(input('Enter the item day :'))

item_date_month = float(input('Enter the item month:'))

item_date_year = float(input('Enter the item year :'))

delivery_date_day = float(input('Enter the delivery day :'))

delivery_date_month = float(input('Enter the delivery month :'))

delivery_date_year = float(input('Enter the delivery year :'))

Enter the Quantity Ton value :1.2563
enter the cusomer value25462
Enter the country code :24
Enter the  status :3
Enter the item type value :5
Enter the application value :2.35646
Enter the thickness value :2.2696
Enter the width value :1.26656
Enter the product ref value :454666
Enter the item day :13
Enter the item month:6
Enter the item year :2020
Enter the delivery day :1
Enter the delivery month :9
Enter the delivery year :2021


In [59]:
df.loc[10000]

quantity_tons          4.385848e-01
customer               3.016038e+07
country                7.800000e+01
item_type              5.000000e+00
application            3.713572e+00
thickness             -3.566749e-01
width                  7.313220e+00
product_ref            6.119930e+05
item_date_day          1.900000e+01
item_date_month        3.000000e+00
item_date_year         2.021000e+03
delivery_date_day      1.000000e+00
delivery_date_month    7.000000e+00
delivery_date_year     2.021000e+03
status                 2.000000e+00
selling_price          5.975038e+00
Name: 10000, dtype: float64

In [60]:
x = df.loc[1000]
x.values.tolist()

[1.7855886456323182,
 30205825.0,
 25.0,
 5.0,
 2.302585092994046,
 1.0986122886681098,
 7.182352111885263,
 1693867550.0,
 31.0,
 3.0,
 2021.0,
 1.0,
 10.0,
 2021.0,
 1.0,
 7.027314514039777]

In [61]:
selling_price = model.predict([[4.571440618970477,
 30271289.0,
 28.0,
 5.0,
 2.70805020110221,
 1.791759469228055,
 7.313220387090301,
 1670798778.0,
 31.0,
 3.0,
 2021.0,
 1.0,
 9.0,
 2021.0,
 1.0]])

import numpy as np

f'predicted selling price : {(np.exp(selling_price))}' f'actual price {np.exp(6.880384082186005)}'

'predicted selling price : [973.]actual price 972.9999999999999'

# Download Pickle Model

In [64]:
with open('regression_model.pkl', 'wb') as f:
    pickle.dump(model, f)
     

In [65]:
with open('regression_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Prediction

In [69]:
sp = [np.log(54.1511386169278),30156308,28,5,np.log(10),np.log(2),np.log(1500),1670798778,1,4,2021,1,7,2021,1]

In [70]:
sell = model.predict([sp])

In [71]:
np.exp(sell)

array([854.])